In [ ]:
!pip install keras_cv

In [ ]:
!pip install keras_core

In [ ]:
!pip install haversine

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import numpy as np
import scipy as sp
import IPython
import sklearn
from haversine import haversine
from sklearn.model_selection import train_test_split
import statistics


def errors_computation(ground_truth, predictions):
	distances = list()
	for i in range(0,len(ground_truth)):
		ground_truth_list = ground_truth[i].tolist()
		predict_list = predictions[i].tolist()
		h= haversine(tuple(ground_truth_list),tuple(predict_list))*1000  # multiplying by 1000 to transform from Km to m
		distances.append(h)
	return distances

def haversine_mean_error_computation(ground_truth, predictions):
	distances = errors_computation(ground_truth, predictions)
	return statistics.mean(distances)


def haversine_error_statistics_computation(ground_truth, predictions,statistical_metric='mean',percentile=50):
	distances = errors_computation(ground_truth, predictions)
	if statistical_metric=="mean":
		return statistics.mean(distances)
	elif statistical_metric=="median":
		return statistics.median(distances)
	elif statistical_metric=="percentile" and (percentile>=0 or percentile<=100):
		return np.percentile(distances,percentile)
	else:
		return statistics.mean(distances)

In [ ]:
#Creating a squeeze and excitation block
import keras_cv
import tensorflow as tf
import numpy as np
from keras.layers import Activation
from keras_cv.layers import SqueezeAndExcite2D
from keras import backend as K

def SqueezeExcitation(x):
  layer = keras_cv.layers.SqueezeAndExcite2D(8, bottleneck_filters=None,
  squeeze_activation="relu", excite_activation="sigmoid" )
  return layer(x)

In [ ]:
# Creating arusha (leakrelu) activation function by calling leakrelu keras layer
from keras.layers import Activation
from keras import backend as K
import tensorflow as tf
import numpy as np

def arusha(x):
  layer = tf.keras.layers.LeakyReLU(alpha=0.3)
  return layer(x)


In [ ]:
#Creating activation function called sinP
import tensorflow as tf
import numpy as np
from keras.layers import Activation
from keras import backend as K

def sinP(x):
  return tf.math.sin(x) + 1.5*x


In [ ]:
# Creating Kiazi (ELU) activation function by calling elu keras layer
import tensorflow as tf
import numpy as np
from keras.layers import Activation
from keras import backend as K

def kiazi(x):
  layer=tf.keras.layers.ELU(alpha=1.0)
  return layer(x)

In [ ]:
import time
import numpy as np
import tensorflow as tf
import random
import pandas as pd
import math
import keras
import keras_cv
import matplotlib.pyplot as plt
import os
from keras_cv.layers import SqueezeAndExcite2D
from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense, Dropout,Activation,BatchNormalization
from keras.layers import Activation
from keras.optimizers import Adam
#from keras.wrappers.scikit_learn import KerasRegressor
from keras.callbacks import Callback, TensorBoard, ModelCheckpoint, EarlyStopping
from keras import regularizers
from keras.models import Model, load_model
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn import preprocessing
from keras.layers import Input, Dense, Flatten, Lambda, Conv2D, Concatenate, Add, Multiply, Reshape, Dropout, Conv3D, LeakyReLU, AveragePooling2D
from keras import backend as K
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.callbacks import ReduceLROnPlateau
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras import layers
from keras import models
from keras import Input
from sklearn.metrics import r2_score
from sklearn.decomposition import PCA
from keras.layers import GlobalAveragePooling2D, Reshape, Dense, Input

SelebeaSamples = [130429]

def exponential_scheme(x,minimum_value,a=60):
	positive_x= x-minimum_value
	numerator = np.exp(positive_x.div(a))
	denominator = np.exp(-minimum_value/a)
	exponential_x = numerator/denominator
	exponential_x = exponential_x * 1000  #facilitating calculations
	transformed_x = exponential_x
	return transformed_x

def powed_scheme(x,minimum_value,b=1.1):
	positive_x= x-minimum_value
	numerator = positive_x.pow(b)
	denominator = (-minimum_value)**(b)
	powed_x = numerator/denominator
	transformed_x = powed_x
	return transformed_x

def positive_scheme(x,minimum_value):
  positive_x = x-minimum_value
  transformed_x = positive_x
  return transformed_x

def normalized_scheme(x, minimum_value):
  positive_x = x-minimum_value
  normalized_x = positive_x/(-minimum_value)
  transformed_x = normalized_x
  return transformed_x


# reading the data
file = pd.read_csv("/content/drive/My Drive/LORAWAN_DATASETS/lorawan_antwerp_2019_dataset.csv")
albert = pd.read_csv("/content/drive/My Drive/LORAWAN_DATASETS/New2019Dataset.csv")
columns = file.columns
columns_02 = albert.columns

z = albert[columns_02[1:]]
z = z.join(file[columns[75:]])

#shuffling the dataset with 42 seeds
sele = pd.DataFrame(z)
np.random.seed(42)
np.random.shuffle(sele.values)

#Splitting the dataset to get training data and training labels
columns_03 = sele.columns
x = sele[columns_03[:45]]
y = sele[columns_03[45:]]

# Adjusting the minimum_value
x = x.replace(-200,200)
minimum_value = x.min().min() - 1
x = x.replace(200,minimum_value)
print('minimum_value')
print(minimum_value)

transformed_x = powed_scheme(x,minimum_value)
#transformed_x = exponential_scheme(x,minimum_value)
#transformed_x = positive_scheme(x,minimum_value)
#transformed_x = normalized_scheme(x,minimum_value)


#Defining my custom data split
i = 0
for i, SelebeaSample in enumerate(SelebeaSamples):
	# Training size
  trainings_size = 0.7                     # 70% training set
  validation_size = 0.15                     # 15% validation set
  test_size = 0.15                          # 15% test set

  data=transformed_x

  print(SelebeaSample)

  x_train = data[:int(trainings_size*SelebeaSample)] # first 70% of the data
  x_val = data[int(trainings_size*SelebeaSample):int((trainings_size + validation_size) * SelebeaSample)]
  x_test = data[-int(test_size * SelebeaSample):] # last 15% of the data

  y_train = y[:int(trainings_size*SelebeaSample)] # first 70% of the data
  y_val = y[int(trainings_size*SelebeaSample):int((trainings_size + validation_size) * SelebeaSample)]
  y_test = y[-int(test_size * SelebeaSample):] # last 15% of the data


'''Scaling data values using MinMaxScaler to ensure that the data values are within a fixed range
and contributes equally to the analysis (MinMaxScaler is useful when the data has a bounded range
or when the distribution is not Gaussian)'''

scaler = preprocessing.StandardScaler().fit(x_train)
x_train = scaler.transform(x_train)
x_val = scaler.transform(x_val)
x_test = scaler.transform(x_test)

scaler_y = preprocessing.MinMaxScaler().fit(y_train)
y_train = scaler_y.transform(y_train)
y_val = scaler_y.transform(y_val)
y_test = scaler_y.transform(y_test)

# Make an instance of the Model
pca = PCA()

pca.fit(x_train)
# Determining the number of components PCA has fitted
N=pca.n_components_

# Calculate Variance Explained
var_exp = pca.explained_variance_ratio_

# Calculate Cumulative Variance Explained
cum_var_exp = np.cumsum(var_exp)

print(var_exp)
#print(cum_var_exp)
print(N)

# Applying the mapping (Transform) to the training, validation and test set
x_train = pca.transform(x_train)
x_val = pca.transform(x_val)
x_test = pca.transform(x_test)

print(x_train.shape)
print(x_val.shape)
print(x_test.shape)

a=x_train.shape[0]
b=x_val.shape[0]
c=x_test.shape[0]

print(y_train.shape)
print(y_val.shape)
print(y_test.shape)

#changing shape of the input data to be compatible with conv2D layers
x_train=x_train.reshape((a,1,45,1))
x_val=x_val.reshape((b,1,45,1))
x_test=x_test.reshape((c,1,45,1))

#cumulative explained variance ratio as a function of the number of components:

#plt.plot(np.cumsum(pca.explained_variance_ratio_))
#plt.xlabel('number of components')
#plt.ylabel('cumulative explained variance')

dropout = 0.15
#l2 = 0.00
learning_rate = 0.001
epochs = 150
batch_size= 512
patience = 100

''' Defining the architecture of the CNN model '''

input_tensor = Input(shape=(1, 45, 1))

branch_a = Conv2D(8, (1, 1), padding='same', kernel_regularizer=regularizers.l1(0.01))(input_tensor)
branch_a = SqueezeExcitation(branch_a)
branch_a = arusha(branch_a)
branch_a = Dropout(dropout)(branch_a)

branch_b = Conv2D(8, (1, 1), padding='same')(input_tensor)
branch_b = SqueezeExcitation(branch_b)
branch_b = arusha(branch_b)
branch_b = Dropout(dropout)(branch_b)
branch_b = Conv2D(8, (1, 1), padding='same', kernel_regularizer=regularizers.l1(0.01))(branch_b)
branch_b = SqueezeExcitation(branch_b)
branch_b = arusha(branch_b)

branch_c = Conv2D(8, (1, 1), padding='same')(input_tensor)
branch_c = SqueezeExcitation(branch_c)
branch_c = arusha(branch_c)
branch_c = Dropout(dropout)(branch_c)
branch_c = Conv2D(8, (1, 1), padding='same')(branch_c)
branch_c = SqueezeExcitation(branch_c)
branch_c = arusha(branch_c)
branch_c = Dropout(dropout)(branch_c)
branch_c = Conv2D(8, (1, 1), padding='same', kernel_regularizer=regularizers.l1(0.01))(branch_c)
branch_c = SqueezeExcitation(branch_c)
branch_c = arusha(branch_c)

output = Concatenate()([branch_a, branch_b, branch_c])

output_tensor= Flatten()(output)
output_tensor = Dense(512, activation='relu')(output_tensor)
output_tensor = Dense(256, activation='relu')(output_tensor)
output_tensor = Dense(128, activation='relu')(output_tensor)
output_tensor = Dense(64, activation='relu')(output_tensor)
output_tensor = Dense(32, activation='relu')(output_tensor)
output_tensor = Dense(2, activation='linear')(output_tensor)
MyModel = Model(inputs=input_tensor, outputs=output_tensor)
MyModel.compile(optimizer=Adam(learning_rate=learning_rate), loss='mae', metrics=['mae'])
MyModel.summary()

cb =[EarlyStopping(monitor='val_loss', mode='min', patience=patience, verbose =1, restore_best_weights=True)]
dlr = [keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10)]
mc = ModelCheckpoint('bestmodels/best_model_lorawan_' + str(SelebeaSample) + '.h5', monitor='val_mae', mode='min', verbose=1, save_best_only=True)
#history = MyModel.fit(x_train, y_train,validation_data=(x_val, y_val),epochs=epochs, batch_size=batch_size, verbose=1, callbacks= [cb,mc])
history = MyModel.fit(x_train, y_train,validation_data=(x_val, y_val),epochs=epochs, batch_size=batch_size, verbose=1, callbacks= [cb,dlr,mc])

# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Train and Validation Errors During Model Training')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper right')
plt.savefig("/content/drive/My Drive/RESULTS/training_curves.png")
plt.show()

MyModel = load_model('bestmodels/best_model_lorawan_' + str(SelebeaSample)  + '.h5')
y_predict = MyModel.predict(x_test, batch_size=batch_size)
y_predict_in_val = MyModel.predict(x_val, batch_size=batch_size)
y_predict_in_train = MyModel.predict(x_train, batch_size=batch_size)


y_predict = scaler_y.inverse_transform(y_predict)
pd.DataFrame(y_predict).to_csv("/content/drive/My Drive/RESULTS/predicted_y_values.csv")



y_predict_in_train = scaler_y.inverse_transform(y_predict_in_train)
y_predict_in_val = scaler_y.inverse_transform(y_predict_in_val)
y_train = scaler_y.inverse_transform(y_train)
y_val = scaler_y.inverse_transform(y_val)

y_test = scaler_y.inverse_transform(y_test)
pd.DataFrame(y_test).to_csv("/content/drive/My Drive/RESULTS/true_y_values.csv")


#Calculating R2 score
r2 = r2_score(y_test, y_predict)
print('r2 score for this model is', r2)

print("Train set mean error: {:.2f}".format(haversine_error_statistics_computation(y_predict_in_train, y_train,'mean')))
print("Train set median error: {:.2f}".format(haversine_error_statistics_computation(y_predict_in_train, y_train,'median')))
print("Train set 75th perc error: {:.2f}".format(haversine_error_statistics_computation(y_predict_in_train, y_train,'percentile',75)))
print("Val set mean error: {:.2f}".format(haversine_error_statistics_computation(y_predict_in_val, y_val,'mean')))
print("Val set median error: {:.2f}".format(haversine_error_statistics_computation(y_predict_in_val, y_val,'median')))
print("Val set 75th perc.  error: {:.2f}".format(haversine_error_statistics_computation(y_predict_in_val, y_val,'percentile',75)))
print("Test set mean error: {:.2f}".format(haversine_error_statistics_computation(y_predict, y_test,'mean')))
print("Test set median error: {:.2f}".format(haversine_error_statistics_computation(y_predict, y_test,'median')))
print("Test set  75th perc. error: {:.2f}".format(haversine_error_statistics_computation(y_predict, y_test,'percentile',75)))

errors = errors_computation(y_predict,y_test)
pd.DataFrame(errors).to_csv("/content/drive/My Drive/RESULTS/errors.csv")
print("end of experiment!!!")

